[link text](https://towardsdatascience.com/extracting-feature-importances-from-scikit-learn-pipelines-18c79b4ae09a)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 1.0 Importing Libraries and dataset :

### 0.0 Adding boundaries to the dataframes

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

### 0.1 Adding Multiple outputs to the output cell : 

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 0.2 Horizantal display in output cell : 

In [0]:
# For easier understanding 

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

### 0.3 Importing relevant libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()

get_ipython().magic(u'matplotlib inline')
mpl.rcParams['figure.figsize'] = (10.0, 5)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

! pip install eli5
import eli5

**The features and target labels are in separate CSV files so I am also using the pandas merge function to combine them into one data frame.**

In [6]:
train_values = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Algorithms/Feature Importance From Pipelines with eli5/train_values.csv')
train_labels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Algorithms/Feature Importance From Pipelines with eli5/train_labels.csv')
train_values.head(2)
train_labels.head(2)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


,building_id,damage_grade
0,802906,3
1,28830,2


In [10]:
train_values.shape
train_labels.shape

(260601, 39)

(260601, 2)

In [7]:
# Merging train_values and train_labels based on building id : 

train_data = train_values.merge(train_labels,on= 'building_id', how = 'left')
train_data.head(2)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2


## 2.0 Data Wrangling : 

In [8]:
train_data.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [0]:
# Dropping building_id as its not required : 
train_data = train_data.drop(columns = 'building_id')

In [0]:
# Extracting the numerical and categorical features column names : 

numerical_features = train_data.select_dtypes(include = ['int64', 'float64']).drop(['damage_grade'], axis = 1).columns
categorical_features = train_data.select_dtypes(include = ['object']).columns

In [11]:
# Splitting data into train and test : 

X = train_data.drop('damage_grade', axis = 1)
y = train_data['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Size of train data : ' ,X_train.shape)
print('Size of train_data_labels : ', y_train.shape)
print('Size of test data', X_test.shape)
print('Size of test_data_labels : ', y_test.shape)

Size of train data :  (208480, 38)
Size of train_data_labels :  (208480,)
Size of test data (52121, 38)
Size of test_data_labels :  (52121,)


## 3.0 Pipeline Construction : 

Steps : 

* 1.Construct a simple pipeline which will chain together the preprocessing and model fitting steps.
* 2.Adding an imputer for any missing values.

In [12]:
# The code below constructs a pipeline that imputes any missing values, applies a standard scaler to the numerical features, 
# converts any categorical features into numerical and then fits a classifier.

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',  LogisticRegression(class_weight='balanced', random_state=0))])
    
model = pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
# Checking the performance of the model : 

target_names = y_test.unique().astype(str)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           2       0.39      0.68      0.50      5170
           3       0.67      0.38      0.48     29487
           1       0.45      0.68      0.54     17464

    accuracy                           0.51     52121
   macro avg       0.50      0.58      0.51     52121
weighted avg       0.57      0.51      0.50     52121



## 4.0 Feature Importance with eli5 : 

**ELI5 needs to know all feature names in order to construct feature importances.**

By applying one-hot encoding to the categorical variables in the pipeline we are introducing a number of new features.We therefore first need to extract these feature names and append them to the known list of numerical features. The code below uses the ‘named_steps’ function built into scikit-learn pipelines to do this.

In [0]:
onehot_columns = list(pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot'].get_feature_names(input_features=categorical_features))
numeric_features_list = list(numerical_features)
numeric_features_list.extend(onehot_columns)


In [16]:
# To extract the feature importances we then simply need to run this line of code. : 

eli5.explain_weights(pipe.named_steps['classifier'], top=50, feature_names=numeric_features_list)

Explanation(estimator="LogisticRegression(C=1.0, class_weight='balanced', dual=False,\n                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,\n                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,\n                   warm_start=False)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='foundation_type_i', weight=0.4702548885842672, std=None, value=None), FeatureWeight(feature='position_s', weight=0.32393638247891243, std=None, value=None), FeatureWeight(feature='other_floor_type_s', weight=0.3091401656965344, std=None, value=None), FeatureWeight(feature='ground_floor_type_v', weight=0.29864898587586086, std=None, value=None), FeatureWeight(feature='roof_type_x', weight=0.25155701789841145, std=None, value=None), FeatureWeight(feature='legal_ownership_status_a', weight=0.21150190393073454, std=None, value=None), FeatureWeight(feature='foundation_type_w', weight=0.16332764322829887, std=None, value=None), FeatureWeight(feature='geo_level_1_id', weight=0.15532220615517178, std=None, value=None), FeatureWeight(feature='foundation_type_h', weight=0.14292188740440667, std=None, value=None), FeatureWeight(feature='plan_configuration_c', weight=0.14132326690395874, std=None, value=None), FeatureWeight(feature='plan_configuration_a', weight=0.13486888247064585, std=None, value=None), FeatureWeight(feature='plan_configuration_n', weight=0.11517235206261264, std=None, value=None), FeatureWeight(feature='land_surface_condition_t', weight=0.11342562956449506, std=None, value=None), FeatureWeight(feature='has_superstructure_timber', weight=0.11143686642137049, std=None, value=None), FeatureWeight(feature='has_superstructure_cement_mortar_brick', weight=0.11094629741013103, std=None, value=None), FeatureWeight(feature='legal_ownership_status_v', weight=0.11004385516778316, std=None, value=None), FeatureWeight(feature='plan_configuration_o', weight=0.10217821181792519, std=None, value=None), FeatureWeight(feature='has_superstructure_rc_engineered', weight=0.1016365882457494, std=None, value=None), FeatureWeight(feature='has_secondary_use_hotel', weight=0.08665996206894425, std=None, value=None), FeatureWeight(feature='plan_configuration_u', weight=0.07967820746191197, std=None, value=None), FeatureWeight(feature='has_secondary_use_rental', weight=0.07964118419954648, std=None, value=None), FeatureWeight(feature='plan_configuration_s', weight=0.07575433890974174, std=None, value=None), FeatureWeight(feature='has_superstructure_rc_non_engineered', weight=0.059956574678358056, std=None, value=None), FeatureWeight(feature='has_secondary_use_other', weight=0.05552722083223518, std=None, value=None), FeatureWeight(feature='has_superstructure_other', weight=0.04151289510454484, std=None, value=None), FeatureWeight(feature='position_j', weight=0.040420698538564816, std=None, value=None), FeatureWeight(feature='geo_level_2_id', weight=0.03895980062799957, std=None, value=None)], neg=[FeatureWeight(feature='foundation_type_r', weight=-0.50915249149195, std=None, value=None), FeatureWeight(feature='has_superstructure_mud_mortar_stone', weight=-0.3432448802985642, std=None, value=None), FeatureWeight(feature='legal_ownership_status_w', weight=-0.33741475139972604, std=None, value=None), FeatureWeight(feature='plan_configuratio